In [14]:
import astropy as ast
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon

import datetime
from datetime import datetime
import pandas as pd
import numpy as np

In [ ]:
bodies = solar_system_ephemeris

In [ ]:
bodies.bodies

In [4]:
# to get the position of the moon, we need to make a time object
example_time = datetime(1998, 8, 8, 2, 45, 37)
future_time = datetime(1999, 8, 8, 2, 45, 37)
future_time = ast.time.Time(future_time)
moon_time = ast.time.Time(example_time)

#inpputing into the get_moon function to get the position
moon = get_moon(moon_time)
moon

<SkyCoord (GCRS: obstime=1998-08-08 02:45:37, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, km)
    (317.74514507, -14.75479743, 372805.1468126)>

In [ ]:
# listing different locations in database
EarthLocation.get_site_names()

In [ ]:
# Trying other planets with a location
location = EarthLocation.of_site('Sacramento Peak')


with solar_system_ephemeris.set('builtin'):
    mars = get_body('mars', moon_time, location)

mars
# this took a while to run ~15 seconds.. 
# may be a problem for generating data

In [ ]:
# trying with cartesian coords
# no need for earth location because it is not relative to earth
with solar_system_ephemeris.set('builtin'):
    mars_cart = get_body_barycentric('mars', moon_time)

mars_cart

In [ ]:
# trying to understand the skycoord object
mars

In [ ]:
mars.obsgeovel

In [ ]:
mars.info

In [ ]:
mars.to_string('decimal')

In [ ]:
mars.distance.to_string()

In [ ]:
mars.representation_type

In [ ]:
mars.shape

In [ ]:
# this method computes thes new position of the object given two optional inputs: a new time or 
# time delta for the amount of time to evolve the postion of the source

# you must input one of them with a velocity.. don't know if this is going to be useful

mars.apply_space_motion(new_obstime = future_time)

In [ ]:
# Computes coordinates at the given offset from this coordinate.

# Parameters
# position_angle : position_angle of offset

# separation : offset angular separation

# Returns
# newpoints : SkyCoord
# The coordinates for the location that corresponds to offsetting by the given position_angle and separation.

# TL;DR
# given new angles returns new position

mars.directional_offset_by(45, 24)

In [ ]:
# obvious: gets constellation that it's apart of
mars.get_constellation()

In [ ]:
# Computes the on-sky position angle (East of North) between this SkyCoord and another.

mars.position_angle(moon)

In [ ]:
# Computes on-sky separation between this coord and another

mars.separation(moon)

In [ ]:
mars.separation_3d(moon)

In [ ]:
#Returns the sky offset frame with this SkyCoord at the origin.
mars.skyoffset_frame()

In [ ]:
# Default 
mars.to_string('decimal')

In [ ]:
mars.to_string('dms')

In [ ]:
mars.to_string('hmsdms')

### Generating Data From Planets 
Mercury, Venus, Mars, Jupiter and Saturn

In [13]:
# This cell takes forever btw (>3 hours lol)
time = 
# helper_function
def get_spherical(body):
    # Returns (theta, phi, r) in (deg, deg, AU)
    # given Skycoord object
    
    angles = [float(i) for i in body.to_string().split(' ')]
    
    # removing the units and converting to float
    dist = float(mars.distance.to_string()[:-3])
    
    full_tuple = (angles[0], angles[1], dist)
    
    return full_tuple

# creating datetime objects for every day at 12am for 150 years
# we're trying to predict the locations at other times during the day
times = pd.date_range(start="1870-01-01-00-00-00", end="2020-01-01-00-00-00", freq = 'W')

celestial_bodies = pd.DataFrame()
# iterating over times
for time in times:
    # opening and instantiating the planets and heavenly bodies
    loc = EarthLocation.of_site('medicina')
    time = moon_time
    with solar_system_ephemeris.set('builtin'):
        mercury = get_body('mercury', time, loc)
        venus = get_body('venus', time, loc)
        mars = get_body('mars', time, loc)
        jupiter = get_body('jupiter', time, loc)
        saturn = get_body('saturn', time, loc)
        moon = get_body('moon', time, loc)
        sun = get_body('sun', time, loc)
    
    
    # I guess I coulda made temp just a series and appended it to the 
    # dataframe to make it faster but this seemed a simpler way to go
    
    # creating a dataframe of tuples
    # (theta, phi, r) in (deg, deg, AU)
    temp = pd.DataFrame({'Time' : time, 
                        'Mercury' : get_spherical(mercury),
                        'Venus' : get_spherical(venus),
                        'Mars' : get_spherical(mars),
                        'Jupiter' : get_spherical(jupiter),
                        'Saturn' : get_spherical(saturn),
                        'Moon' : get_spherical(moon),
                        'Sun' : get_spherical(sun)})
    
    celestial_bodies = pd.concat([celestial_bodies, temp], ignore_index=True)
    


celestial_bodies.head()



,Time,Mercury,Venus,Mars,Jupiter,Saturn,Moon,Sun
0,1998-08-08 02:45:37,146.207000,115.486000,113.692000,358.157000,32.249000,317.172000,137.862000
1,1998-08-08 02:45:37,8.469830,21.522600,22.541800,-2.380330,10.312800,-15.529100,16.217400
2,1998-08-08 02:45:37,2.462631,2.462631,2.462631,2.462631,2.462631,2.462631,2.462631
3,1998-08-08 02:45:37,146.207000,115.486000,113.692000,358.157000,32.249000,317.172000,137.862000
4,1998-08-08 02:45:37,8.469830,21.522600,22.541800,-2.380330,10.312800,-15.529100,16.217400


In [15]:
celestial_bodies.shape

(164361, 8)